In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [75]:
import tensorflow as tf

In [76]:
path_to_file = r'shakespeare.txt'

In [77]:
with open(path_to_file, 'r') as file:
    text = file.read()

In [78]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [79]:
vocab = sorted(set(text))

In [80]:
# vocab

In [81]:
len(vocab)

84

In [82]:
char_to_ind = {char:ind for ind, char in enumerate(vocab)}

In [83]:
# ind_to_char = {ind:char for ind, char in enumerate(vocab)}

In [84]:
ind_to_char = np.array(vocab)

In [85]:
encoded_text = np.array([char_to_ind[c] for c in text])  ## character to number

In [86]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

In [87]:
encoded_text.shape

(5445609,)

In [88]:
text[:500]

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bu"

In [89]:
len('From fairest creatures we desire increase')

41

In [90]:
len('''From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease''')

130

#### creatig batches

In [91]:
seq_len = 120

In [92]:
total_num_seq = len(text) // (seq_len +1)

In [93]:
total_num_seq

45005

In [94]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [95]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [96]:
# for item in char_dataset.take(500):
#     print(ind_to_char[item.numpy()])

In [97]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [98]:
def create_sequence_target(seq):
    input_text = seq[:-1] #hello my nam
    target_text = seq[1:] # ello my name
    
    return input_text, target_text

In [99]:
dataset = sequences.map(create_sequence_target)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [100]:
for input_text, target_text in dataset.take(1):
    print(input_text.numpy())
    print("".join(ind_to_char[input_text.numpy()]))
    print('___________________________________')
    print(target_text.numpy())
    print("".join(ind_to_char[target_text.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But
___________________________________
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never di

In [101]:
batch_size = 128

In [102]:
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [103]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int32, tf.int32)>

##### creating model

In [104]:
vocab_size = len(vocab)

In [105]:
embed_dim = 64

In [116]:
rnn_neurons = 1026

In [107]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [108]:
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [109]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GRU

In [110]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size,None]))
    model.add(GRU(rnn_neurons, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    
    model.add(Dense(vocab_size))
    
    model.compile(optimizer='adam', loss=sparse_cat_loss)
    
    return model

In [111]:
model = create_model(vocab_size=vocab_size,
                    embed_dim=embed_dim,
                    rnn_neurons=rnn_neurons,
                    batch_size=batch_size)

In [112]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (128, None, 64)           5376      
_________________________________________________________________
gru_2 (GRU)                  (128, None, 1024)         3348480   
_________________________________________________________________
dense_2 (Dense)              (128, None, 84)           86100     
Total params: 3,439,956
Trainable params: 3,439,956
Non-trainable params: 0
_________________________________________________________________


#### train model

random text geenration for one sample

In [56]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_prediction = model(input_example_batch)

In [58]:
example_batch_prediction.shape

TensorShape([128, 120, 84])

In [59]:
example_batch_prediction[0]

<tf.Tensor: shape=(120, 84), dtype=float32, numpy=
array([[-0.00076059, -0.00338398,  0.00185325, ...,  0.00092757,
        -0.00023945, -0.00139248],
       [ 0.00030299,  0.00097693,  0.01192942, ..., -0.0046382 ,
        -0.00505411, -0.00092662],
       [-0.00361676,  0.00446928,  0.00663559, ..., -0.00194609,
         0.00337014,  0.00439108],
       ...,
       [-0.00524939, -0.00503995,  0.00365616, ..., -0.00453976,
         0.00734555, -0.00668871],
       [-0.00520948, -0.00773752, -0.00819679, ...,  0.00246347,
         0.00465396, -0.00449445],
       [-0.00449074, -0.00770512, -0.00284968, ...,  0.00144689,
         0.00122634, -0.00473004]], dtype=float32)>

In [60]:
sample_indices = tf.random.categorical(example_batch_prediction[0], num_samples=1)

In [63]:
sample_indices = tf.squeeze(sample_indices, axis=-1).numpy()
sample_indices

array([71, 22, 44, 39, 53, 31, 24, 37,  2,  4, 36, 31, 75, 33, 81, 76, 12,
       26, 71, 68, 78, 47, 52, 80, 58, 32, 41, 53, 28, 53, 45,  6,  9, 65,
       69, 73,  5, 53, 34,  8, 40,  4, 53,  6, 30, 46, 39,  4, 38, 43, 41,
       46, 10, 30, 56, 47,  1, 50, 48, 66,  8, 40, 52, 51, 78, 30, 75, 53,
       52,  3, 36, 54, 69, 43, 73, 46, 75, 56, 41, 38, 64, 28, 80, 75,  5,
       28, 64,  0, 19, 15, 25, 52, 22, 13,  3, 34, 40,  5, 42, 23, 73, 69,
       74, 78, 22, 62, 69, 75, 71, 19, 24, 78,  2, 34, 68, 10, 16, 60, 70,
       63], dtype=int64)

In [65]:
ind_to_char[sample_indices]  ### random charcter predcitons

array(['p', ';', 'S', 'N', ']', 'F', '>', 'L', '!', '&', 'K', 'F', 't',
       'H', 'z', 'u', '1', 'A', 'p', 'm', 'w', 'V', '[', 'y', 'c', 'G',
       'P', ']', 'C', ']', 'T', '(', '-', 'j', 'n', 'r', "'", ']', 'I',
       ',', 'O', '&', ']', '(', 'E', 'U', 'N', '&', 'M', 'R', 'P', 'U',
       '.', 'E', 'a', 'V', ' ', 'Y', 'W', 'k', ',', 'O', '[', 'Z', 'w',
       'E', 't', ']', '[', '"', 'K', '_', 'n', 'R', 'r', 'U', 't', 'a',
       'P', 'M', 'i', 'C', 'y', 't', "'", 'C', 'i', '\n', '8', '4', '?',
       '[', ';', '2', '"', 'I', 'O', "'", 'Q', '<', 'r', 'n', 's', 'w',
       ';', 'g', 'n', 't', 'p', '8', '>', 'w', '!', 'I', 'm', '.', '5',
       'e', 'o', 'h'], dtype='<U1')

In [66]:
##training

In [67]:
epochs= 30

In [69]:
model.fit(dataset, epochs=epochs)

In [ ]:
model.save('shakespeare_gen.h5')

In [117]:
from tensorflow.keras.models import load_model

In [118]:
model = create_model(vocab_size=vocab_size,
                    embed_dim=embed_dim,
                    rnn_neurons=rnn_neurons,
                    batch_size=1)

In [119]:
model.load_weights(r'C:\Users\user\Downloads\Compressed\FINAL_TF2_FILES\TF_2_Notebooks_and_Data\06-NLP-and-Text-Data\shakespeare_gen.h5')

In [121]:
model.build(tf.TensorShape([1,None]))

In [122]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_4 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [132]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
    '''
    model: Trained Model to Generate Text
    start_seed: Intial Seed text in string form
    gen_size: Number of characters to generate

    Basic idea behind this function is to take in some seed text, format it so
    that it is in the correct shape for our network, then loop the sequence as
    we keep adding our own predicted characters. Similar to our work in the RNN
    time series problems.
    '''

    # Number of characters to generate
    num_generate = gen_size

    # Vecotrizing starting seed text
    input_eval = [char_to_ind[s] for s in start_seed]

    # Expand to match batch format shape
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty list to hold resulting generated text
    text_generated = []

    # Temperature effects randomness in our resulting text
    # The term is derived from entropy/thermodynamics.
    # The temperature is used to effect probability of next characters.
    # Higher probability == lesss surprising/ more expected
    # Lower temperature == more surprising / less expected

    temperature = temp

    # Here batch size == 1
    model.reset_states()

    for i in range(num_generate):

        # Generate Predictions
        predictions = model(input_eval)

        # Remove the batch shape dimension
        predictions = tf.squeeze(predictions, 0)

        # Use a cateogircal disitribution to select the next character
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted charracter for the next input
        input_eval = tf.expand_dims([predicted_id], 0)

        # Transform back to character letter
        text_generated.append(ind_to_char[predicted_id])

    return (start_seed + ''.join(text_generated))

In [133]:
print(generate_text(model, 'Juliet', gen_size=1000))

Juliet:
  SALERIO. Mistress Antony, being flexheed, the best, Kate to
ANTIPHOLUS OF EPHESUS, and grave me thou ask the mind.
  AJAX. It is a creature now our counsel.
  ROSALINE. Well, have you heard a friend?
  THIRD LORD. Ay, ay some power your guide. Plantagenet sorrow fair.
  SHEPHERD. I will not.
  SECRET. There is no town among myself, but tell them,
    That rich employ'd and was much writ.

                    Enter ACHILLES
  CURTIN, a
                Likest the approaches of the rose,
    He makes the rest sleep with spite of it,
    Rise and well arred words,
    For 'tis a country as his honour
    As is contented so.
  DIOMEDES. May I gash keep you for a monthamblor?
  VOLUMNIA. There twenty you here do forb'd in with
    pokes treesters for our upboint a mother on his weapon. Paund for the
     pigeorse fit for it; at an ill fearing glad,
    Making
    Keeps in, with his against he swear not to her; you love thee not?
  PATROCLUS. You are old, and cried 'bus' to be neces